In [0]:
%%sh
m='201307'
PGPASSWORD='2mHdFW6%#REu' psql -h internal-aa-prod-plproxy-internal-2-31838298.us-east-1.elb.amazonaws.com -U app_bdp_usage_qa -d cohort -p 7432 << EOF 

SELECT device_id,store_id,date,app_id,kpi,estimate,age, gender
  FROM plproxy.execute_select_nestloop(\$proxy\$select device_id,store_id,date,app_id,kpi,estimate,age, gender from ag.app_monthly_1001_1_201802
 \$proxy\$) t (device_id SMALLINT,store_id INT,date DATE,app_id BIGINT,kpi SMALLINT, estimate FLOAT, age SMALLINT, gender SMALLINT)  where app_id= '20600006257774' and kpi=22 order by estimate desc limit 50;

EOF



In [0]:

path="s3://b2c-prod-data-pipeline-unified-usage/unified/app-tech.usage.legacy-ag_app.v1/fact/granularity=weekly/month=2014-02/device_id=2001/store_id=143441/"
# print 'raw data', spark.read.parquet(path).groupBy("kpi").agg(count("kpi")).show()
# print spark.read.parquet(path).groupBy("kpi").agg(count("kpi")).show()
# df1=spark.read.parquet(path).filter("kpi=22").select("kpi","age","app_id","gender").distinct().withColumnRenamed("app_id","raw_app_id")
df1 = spark.read.parquet(path).filter("date='2014-02-22'").select("kpi","age","app_id","gender").distinct().groupBy("kpi").agg(count("kpi")).collect()
print df1
path="s3://b2c-prod-data-pipeline-unified-usage/unified/app-tech.usage.segmented-by-age-gender.v1/fact/granularity=weekly/date=2014-02-22"
df2 = spark.read.parquet(path).filter("device_code='ios-phone' and country_code='US' and est_panel_size is not null").count()
print df2
# df2=spark.read.parquet(path).filter("device_code='ios-phone' and country_code='US' and est_panel_size is not null").withColumnRenamed("app_id","unified_app_id")

# union_df=df1.join(df2, df1["raw_app_id"]==df2["unified_app_id"], how='full').filter("unified_app_id is  null")
# print 'union count', union_df.count()
# union_df.show()
# AssertionError:  raw: 31476 ~ unified data: 25788,device:2001,  store_id:143441 , month: (u'2014-02', u'2014-02-22'), KPI 22



# AssertionError:  raw: 134348 ~ unified data: 134364,device:2001,  store_id:143441 , month: (u'2017-04', u'2017-04-30')

#  AssertionError:  raw: 108809 ~ unified data: 100440,device:2001,  store_id:143441 , month: (u'2015-02', u'2015-02-28')

#                     raw_path="s3://b2c-prod-data-pipeline-unified-usage/unified/app-tech.usage.legacy-ag_app.v1/fact/granularity={raw_granularity}/month={raw_month}/device_id={raw_device_id}/store_id={raw_store_id}/"
#                     unified_path="s3://b2c-prod-data-pipeline-unified-usage/unified/app-tech.usage.segmented-by-age-gender.v1/fact/granularity={unified_granularity}/date={unified_date}/"

# +---+----------+
# |kpi|count(kpi)|
# +---+----------+
# |  1|    134348|
# |  9|    134348|
# |  4|    134268|
# | 10|    134280|
# +---+----------+


In [0]:


path="s3://b2c-prod-data-pipeline-unified-usage/unified/app-tech.usage.legacy-ag_app.v1/fact/granularity=weekly/month=2014-02/device_id=2001/store_id=143441/"
# print 'raw data', spark.read.parquet(path).groupBy("kpi").agg(count("kpi")).show()
# print spark.read.parquet(path).groupBy("kpi").agg(count("kpi")).show()
df1=spark.read.parquet(path).filter("kpi=22 and app_id=307312434 and date= ").show()

In [0]:
%%sh
aws s3 ls s3://b2c-prod-data-pipeline-unified-usage/unified/app-tech.usage.legacy-ag_app.v1/fact/granularity=weekly/month=2014-02/device_id=2001/store_id=143441/
 

aws s3 ls s3://b2c-prod-data-pipeline-unified-usage/unified/app-tech.usage.legacy-ad_app.v1/fact/granularity=monthly/



In [0]:

import unittest
import datetime
from dateutil.relativedelta import relativedelta
from pyspark.sql.utils import AnalysisException
from pyspark.sql.functions import count
from pyspark.sql import Row

kpi_mapping={1:"est_average_active_users", 2: "est_average_session_per_user", 3:"est_average_session_duration", 4:"est_install_penetration", 5:"est_average_active_days", 6:"est_percentage_active_days", 7:"est_average_bytes_per_user" , 8:"est_average_time_per_user", 9:"est_usage_penetration", 10:"est_open_rate",11:"est_total_time", 12:"est_share_of_category_time",14:"est_total_sessions", 15:"est_share_of_category_session", 17:"est_average_bytes_per_session", 18:"est_share_of_category_bytes", 20:"est_percent_of_wifi_total", 21:"est_mb_per_second", 22:"est_panel_size", 23:"est_installs", 24:"est_average_active_users_country_share", 25:"est_installs_country_share", 26:"est_audience_index", 27:"est_audience_percentage"}

ANDROID_COUNTRY_ID_CODES = { 1: 'AU', 2: 'CA', 3: 'CN', 4: 'DE', 5: 'ES', 6: 'FR', 7: 'GB', 8: 'IT', 9: 'JP', 10: 'US', 11: 'BE', 12: 'CH', 13: 'CL', 14: 'ZA', 15: 'VN', 16: 'HK', 17: 'AR', 18: 'BR', 19: 'IN', 20: 'FI', 21: 'ID', 22: 'RU', 23: 'NL', 24: 'MY', 25: 'TR', 26: 'MX', 27: 'KR', 28: 'PL', 29: 'TH', 30: 'TW', 31: 'PH', 32: 'SG', 33: 'EG', 34: 'SE', 35: 'AT', 36: 'CZ', 37: 'HU', 38: 'DK', 39: 'IE', 40: 'IL', 41: 'NZ', 42: 'NO', 43: 'PT', 44: 'RO', 45: 'SK', 46: 'GR', 47: 'BG', 48: 'UA', 49: 'AE', 50: 'KW', 51: 'SA', 52: 'CO', 65: 'LB', 56: 'PE', 80: 'HR', 54: 'PK', 62: 'EC', 73: 'QA', 102: 'MO', 103: 'LU', 53: 'KZ', 1000: 'WW' }


IOS_COUNTRY_ID_CODES={143460:'AU',143455:'CA',143465:'CN',143443:'DE',143454:'ES',143442:'FR',143444:'GB',143450:'IT',143462:'JP',143441:'US',143446:'BE',143459:'CH',143483:'CL',143472:'ZA',143471:'VN',143463:'HK',143505:'AR',143503:'BR',143467:'IN',143447:'FI',143476:'ID',143469:'RU',143452:'NL',143473:'MY',143480:'TR',143468:'MX',143466:'KR',143478:'PL',143475:'TH',143470:'TW',143474:'PH',143464:'SG',143516:'EG',143456:'SE',143445:'AT',143489:'CZ',143482:'HU',143458:'DK',143449:'IE',143491:'IL',143461:'NZ',143457:'NO',143453:'PT',143487:'RO',143496:'SK',143448:'GR',143526:'BG',143492:'UA',143481:'AE',143493:'KW',143479:'SA',143501:'CO',143451:'LU',143497:'LB',143515:'MO',143507:'PE',143494:'HR',143477:'PK',143509:'EC',143498:'QA',0:'WW'}

def get_device_list(device):
    if device=='ios':
        return {"2001":"ios-phone","2002":"ios-tablet"}
    else:
        return {"1001":"android-phone" ,"1002":"android-tablet"}


def last_day_of_month(check_month):
    next_month = check_month.replace(day=28) + datetime.timedelta(days=4)  
    return next_month - datetime.timedelta(days=next_month.day)


def get_month_list():
    result = []
    today = datetime.date(2019, 10, 1) 
    current = datetime.date(2013, 1, 1)    
    while current <= today:
        month_data_raw=datetime.datetime.strftime(current,'%Y-%m')
        month_data_leg_db=datetime.datetime.strftime(current,'%Y%m')
        result.append((month_data_raw, month_data_leg_db,current  ))
        current += relativedelta(months=1)  
    return result


def get_path_date_list(granularity):
    # df_date = spark.read.parquet("s3://b2c-prod-data-pipeline-unified-usage/unified/app-tech.usage.legacy-ag_app.v1/fact/granularity={}/".format(granularity)).select('date').dropDuplicates()
    # collect_date= df_date.collect()
    collect_date=[Row(date=u'2018-12-31'), Row(date=u'2019-08-31'), Row(date=u'2015-02-28'), Row(date=u'2017-04-30'), Row(date=u'2014-08-31'), Row(date=u'2017-10-31'), Row(date=u'2018-10-31'), Row(date=u'2018-05-31'), Row(date=u'2017-08-31'), Row(date=u'2013-09-30'), Row(date=u'2016-09-30'), Row(date=u'2014-06-30'), Row(date=u'2013-07-31'), Row(date=u'2018-06-30'), Row(date=u'2014-07-31'), Row(date=u'2016-12-31'), Row(date=u'2014-09-30'), Row(date=u'2017-05-31'), Row(date=u'2019-06-30'), Row(date=u'2013-05-31'), Row(date=u'2015-03-31'), Row(date=u'2015-11-30'), Row(date=u'2019-10-31'), Row(date=u'2017-06-30'), Row(date=u'2019-02-28'), Row(date=u'2015-06-30'), Row(date=u'2018-04-30'), Row(date=u'2016-05-31'), Row(date=u'2013-03-31'), Row(date=u'2016-03-31'), Row(date=u'2019-09-30'), Row(date=u'2015-12-31'), Row(date=u'2015-01-31'), Row(date=u'2013-01-31'), Row(date=u'2014-02-28'), Row(date=u'2019-03-31'), Row(date=u'2016-08-31'), Row(date=u'2018-02-28'), Row(date=u'2013-06-30'), Row(date=u'2016-07-31'), Row(date=u'2015-10-31'), Row(date=u'2018-03-31'), Row(date=u'2014-01-31'), Row(date=u'2018-09-30'), Row(date=u'2017-07-31'), Row(date=u'2019-04-30'), Row(date=u'2014-05-31'), Row(date=u'2019-01-31'), Row(date=u'2018-08-31'), Row(date=u'2014-04-30'), Row(date=u'2016-01-31'), Row(date=u'2017-12-31'), Row(date=u'2019-05-31'), Row(date=u'2017-09-30'), Row(date=u'2018-11-30'), Row(date=u'2018-01-31'), Row(date=u'2016-06-30'), Row(date=u'2015-04-30'), Row(date=u'2015-05-31'), Row(date=u'2018-07-31'), Row(date=u'2016-02-29'), Row(date=u'2015-09-30'), Row(date=u'2013-12-31'), Row(date=u'2014-12-31'), Row(date=u'2013-08-31'), Row(date=u'2013-04-30'), Row(date=u'2019-07-31'), Row(date=u'2013-02-28'), Row(date=u'2017-01-31'), Row(date=u'2017-11-30'), Row(date=u'2013-11-30'), Row(date=u'2013-10-31'), Row(date=u'2017-02-28'), Row(date=u'2016-11-30'), Row(date=u'2016-04-30'), Row(date=u'2014-03-31'), Row(date=u'2014-11-30'), Row(date=u'2015-07-31'), Row(date=u'2017-03-31'), Row(date=u'2014-10-31'), Row(date=u'2016-10-31'), Row(date=u'2015-08-31')]
    date_list = [(x[0][:7],x[0]) for x in collect_date]
    print date_list
    return date_list
    
    

    
import traceback
def check_ag_data_count(store_id_list, device_id_list, _granularity, date_list):
    t = unittest.TestCase('run')
    for id,country_code in store_id_list.items():
        for device,device_code in device_id_list.items():
            for m in date_list:
                raw_count_with_KPI=''
                # print id, device, m[0] , m[1]
                try:
                    raw_path="s3://b2c-prod-data-pipeline-unified-usage/unified/app-tech.usage.legacy-ag_app.v1/fact/granularity={raw_granularity}/month={raw_month}/device_id={raw_device_id}/store_id={raw_store_id}/"
                    unified_path="s3://b2c-prod-data-pipeline-unified-usage/unified/app-tech.usage.segmented-by-age-gender.v1/fact/granularity={unified_granularity}/date={unified_date}/"
                    raw_path_parse=raw_path.format(raw_device_id=device,raw_store_id=id, raw_month=m[0], raw_granularity=_granularity)
                    raw_count_with_KPI=spark.read.parquet(raw_path_parse).filter("date='{}'".format(m[1])).select("kpi","age","app_id","gender").distinct().groupBy("kpi").agg(count("kpi")).collect()
                    # print raw_count_with_KPI
                except AnalysisException as e: 
                    break
                    # traceback.print_exc()
                # print 'raw count', raw_count
                for row in raw_count_with_KPI:
                    # print 'row _ test', row["kpi"], row["count(kpi)"]
                    unified_path_parse=unified_path.format(unified_date=m[1], unified_granularity=_granularity)
                    unified_count= spark.read.parquet(unified_path_parse).filter("device_code='{}' and country_code='{}'".format(device_code,country_code)).filter("{} is not null".format(kpi_mapping[row["kpi"]])).select(kpi_mapping[row["kpi"]]).count()
                    # print 'unified count' , unified_count
                    t.assertEqual(row["count(kpi)"], unified_count+1, " raw: {} ~ unified data: {},device:{},  store_id:{} , month: {}, KPI {}".format(row["count(kpi)"], unified_count, device, id , m, row["kpi"]))

graularity_list=["monthly","weekly"]
for graularity in graularity_list:
    date_list=get_path_date_list(graularity)
    print graularity
    check_ag_data_count(IOS_COUNTRY_ID_CODES, get_device_list('ios'),graularity,date_list )
    check_ag_data_count(ANDROID_COUNTRY_ID_CODES, get_device_list('android'),graularity, date_list)
print 'pass'



In [0]:

spark.sparkContext.addPyFile("/home/hadoop/bdp/application/libs/python/dependencies.zip")

from aaintdatapipeline.application.app_qa.db_check_v1.constants import pg_settings
from aaintdatapipeline.core.urn import Urn
from aaintdatapipeline.application.app_qa.conf import settings
from aaintdatapipeline.core.loader.plproxy import build_db_settings
from aaplproxy.connection import ClusterConnection
import unittest
import datetime
from dateutil.relativedelta import relativedelta
import psycopg2

ANDROID_COUNTRY_ID_CODES = { 1: 'AU', 2: 'CA', 3: 'CN', 4: 'DE', 5: 'ES', 6: 'FR', 7: 'GB', 8: 'IT', 9: 'JP', 10: 'US', 11: 'BE', 12: 'CH', 13: 'CL', 14: 'ZA', 15: 'VN', 16: 'HK', 17: 'AR', 18: 'BR', 19: 'IN', 20: 'FI', 21: 'ID', 22: 'RU', 23: 'NL', 24: 'MY', 25: 'TR', 26: 'MX', 27: 'KR', 28: 'PL', 29: 'TH', 30: 'TW', 31: 'PH', 32: 'SG', 33: 'EG', 34: 'SE', 35: 'AT', 36: 'CZ', 37: 'HU', 38: 'DK', 39: 'IE', 40: 'IL', 41: 'NZ', 42: 'NO', 43: 'PT', 44: 'RO', 45: 'SK', 46: 'GR', 47: 'BG', 48: 'UA', 49: 'AE', 50: 'KW', 51: 'SA', 52: 'CO', 65: 'LB', 56: 'PE', 80: 'HR', 54: 'PK', 62: 'EC', 73: 'QA', 102: 'MO', 103: 'LU', 53: 'KZ', 1000: 'WW' }


# IOS_COUNTRY_ID_CODES={143460:'AU',143455:'CA',143465:'CN',143443:'DE',143454:'ES'}

IOS_COUNTRY_ID_CODES={143460:'AU',143455:'CA',143465:'CN',143443:'DE',143454:'ES',143442:'FR',143444:'GB',143450:'IT',143462:'JP',143441:'US',143446:'BE',143459:'CH',143483:'CL',143472:'ZA',143471:'VN',143463:'HK',143505:'AR',143503:'BR',143467:'IN',143447:'FI',143476:'ID',143469:'RU',143452:'NL',143473:'MY',143480:'TR',143468:'MX',143466:'KR',143478:'PL',143475:'TH',143470:'TW',143474:'PH',143464:'SG',143516:'EG',143456:'SE',143445:'AT',143489:'CZ',143482:'HU',143458:'DK',143449:'IE',143491:'IL',143461:'NZ',143457:'NO',143453:'PT',143487:'RO',143496:'SK',143448:'GR',143526:'BG',143492:'UA',143481:'AE',143493:'KW',143479:'SA',143501:'CO',143451:'LU',143497:'LB',143515:'MO',143507:'PE',143494:'HR',143477:'PK',143509:'EC',143498:'QA',0:'WW'}

def get_device_list(device):
    if device=='ios':
        return ['2001','2002']
    else:
        return ['1001','1002']

def get_month_list():
    result = []
    today = datetime.date(2019, 10, 1) 
    current = datetime.date(2013, 1, 1)    
    while current <= today:
        month_data_raw=datetime.datetime.strftime(current,'%Y-%m')
        month_data_leg_db=datetime.datetime.strftime(current,'%Y%m')
        result.append((month_data_raw, month_data_leg_db ))
        current += relativedelta(months=1)    
    return result
    
    
def pg_settings(urn, sql, granularity, schema):
    result = {}
    db_settings = build_db_settings(urn, schema)
    connection = ClusterConnection(db_settings)
    master_runner = connection.master_runner
    rows, _ = master_runner.select(sql)
    result[granularity] = rows
    return result


urn = Urn(
    namespace='app-qa.db-validation.v1',
    owner='app_qa'
)

path="s3://b2c-prod-data-pipeline-unified-usage/unified/app-tech.usage.legacy-ag_app.v1/fact/granularity={raw_granularity}/month={raw_month}/device_id={device}/store_id={sid}/"


import traceback
def check_ag_app_dump_data(store_id_list, device_id_list, _granularity):
    t = unittest.TestCase('run')
    for id in store_id_list:
        for device in device_id_list:
            for m in get_month_list():
                # print id, device, m[0] 
                try:
                    weekly_basic = """
                        SELECT app_id_count FROM plproxy.execute_select_nestloop($$select count(app_id) from ag.app_{granularity}_{device_id}_{store_id}_{month} $$) t (app_id_count BIGINT)
                    """.format(device_id=device,store_id=id, month=m[1], granularity=_granularity)
                    granularity_ = _granularity.capitalize()
                    result = pg_settings(urn, weekly_basic, granularity_,'usage')
                    l = [int(x[0]) for x in result[granularity_]]
                    if sum(l)==0:
                        pass
                        # print 'the table is empty  - device:{}  store_id:{} month:{}'.format(device, id, m[0])
                    else:
                        dump_data_path=path.format(device=device, raw_month=m[0], sid=id, raw_granularity=_granularity)
                        dump_count=spark.read.parquet(dump_data_path).count()
                        t.assertEqual(dump_count, sum(l), "{}:{} ~ raw: {} ~ dumped data: {} ".format(granularity_, m[0], dump_count, sum(l)))
                except psycopg2.ProgrammingError : 
                    pass
                    # traceback.print_exc()
                    # print(e)
                    #  print 'the table is not existed - granularity {}, device:{}  store_id:{} month:{}'.format(granularity_, device, id, m[0])
                

check_ag_app_dump_data(IOS_COUNTRY_ID_CODES, get_device_list('ios'),'monthly')
check_ag_app_dump_data(ANDROID_COUNTRY_ID_CODES, get_device_list('android'),'monthly')
check_ag_app_dump_data(IOS_COUNTRY_ID_CODES, get_device_list('ios'),'weekly')
check_ag_app_dump_data(ANDROID_COUNTRY_ID_CODES, get_device_list('android'),'weekly')
print 'pass'



In [0]:

spark.sparkContext.addPyFile("/home/hadoop/bdp/application/libs/python/dependencies.zip")

from aaintdatapipeline.application.app_qa.db_check_v1.constants import pg_settings
from aaintdatapipeline.core.urn import Urn
from aaintdatapipeline.application.app_qa.conf import settings
from aaintdatapipeline.core.loader.plproxy import build_db_settings
from aaplproxy.connection import ClusterConnection
import unittest
import datetime
from dateutil.relativedelta import relativedelta
import psycopg2


kpi_mapping={1:"est_average_active_users", 2: "est_average_session_per_user", 3:"est_average_session_duration", 4:"est_install_penetration", 5:"est_average_active_days", 6:"est_percentage_active_days", 7:"est_average_bytes_per_user" , 8:"est_average_time_per_user", 9:"est_usage_penetration", 10:"est_open_rate", 12:"est_share_of_category_time", 15:"est_share_of_category_session", 17:"est_average_bytes_per_session", 18:"est_share_of_category_bytes", 20:"est_percent_of_wifi_total", 21:"est_mb_per_second", 22:"est_panel_size", 23:"est_installs", 24:"est_average_active_users_country_share", 25:"est_installs_country_share"}


ANDROID_COUNTRY_ID_CODES = { 1: 'AU', 2: 'CA', 3: 'CN', 4: 'DE', 5: 'ES', 6: 'FR', 7: 'GB', 8: 'IT', 9: 'JP', 10: 'US', 11: 'BE', 12: 'CH', 13: 'CL', 14: 'ZA', 15: 'VN', 16: 'HK', 17: 'AR', 18: 'BR', 19: 'IN', 20: 'FI', 21: 'ID', 22: 'RU', 23: 'NL', 24: 'MY', 25: 'TR', 26: 'MX', 27: 'KR', 28: 'PL', 29: 'TH', 30: 'TW', 31: 'PH', 32: 'SG', 33: 'EG', 34: 'SE', 35: 'AT', 36: 'CZ', 37: 'HU', 38: 'DK', 39: 'IE', 40: 'IL', 41: 'NZ', 42: 'NO', 43: 'PT', 44: 'RO', 45: 'SK', 46: 'GR', 47: 'BG', 48: 'UA', 49: 'AE', 50: 'KW', 51: 'SA', 52: 'CO', 65: 'LB', 56: 'PE', 80: 'HR', 54: 'PK', 62: 'EC', 73: 'QA', 102: 'MO', 103: 'LU', 53: 'KZ', 1000: 'WW' }



IOS_COUNTRY_ID_CODES={143460:'AU',143455:'CA',143465:'CN',143443:'DE',143454:'ES',143442:'FR',143444:'GB',143450:'IT',143462:'JP',143441:'US',143446:'BE',143459:'CH',143483:'CL',143472:'ZA',143471:'VN',143463:'HK',143505:'AR',143503:'BR',143467:'IN',143447:'FI',143476:'ID',143469:'RU',143452:'NL',143473:'MY',143480:'TR',143468:'MX',143466:'KR',143478:'PL',143475:'TH',143470:'TW',143474:'PH',143464:'SG',143516:'EG',143456:'SE',143445:'AT',143489:'CZ',143482:'HU',143458:'DK',143449:'IE',143491:'IL',143461:'NZ',143457:'NO',143453:'PT',143487:'RO',143496:'SK',143448:'GR',143526:'BG',143492:'UA',143481:'AE',143493:'KW',143479:'SA',143501:'CO',143451:'LU',143497:'LB',143515:'MO',143507:'PE',143494:'HR',143477:'PK',143509:'EC',143498:'QA',0:'WW'}

def get_device_list(device):
    if device=='ios':
        return {"2001":"ios-phone","2002":"ios-tablet"}
    else:
        return {"1001":"android-phone","1002":"android-tablet"}

def get_month_list():
    result = []
    today = datetime.date(2019, 10, 1) 
    current = datetime.date(2013, 1, 1)    
    while current <= today:
        month_data_raw=datetime.datetime.strftime(current,'%Y-%m')
        month_data_leg_db=datetime.datetime.strftime(current,'%Y%m')
        result.append((month_data_raw, month_data_leg_db,current ))
        current += relativedelta(months=1)    
    return result
    
    
def pg_settings(urn, sql, granularity, schema):
    result = {}
    db_settings = build_db_settings(urn, schema)
    connection = ClusterConnection(db_settings)
    master_runner = connection.master_runner
    rows, _ = master_runner.select(sql)
    result[granularity] = rows
    return result


urn = Urn(
    namespace='app-qa.db-validation.v1',
    owner='app_qa'
)

path="s3://b2c-prod-data-pipeline-unified-usage/unified/app-tech.usage.basic-kpi.v2/fact/granularity={raw_granularity}/date={raw_date}/"


def last_day_of_month(any_day):
    next_month = any_day.replace(day=28) + datetime.timedelta(days=4)  # this will never fail
    return next_month - datetime.timedelta(days=next_month.day)


def get_every_date():
    today = datetime.date(2019, 10, 1) 
    current = datetime.date(2013, 1, 1)    
    delta = today - current
    return [current + datetime.timedelta(days=i) for i in range(delta.days + 1)]


import traceback
def check_au_app_transform_data(store_id_list, device_id_list, _granularity):
    t = unittest.TestCase('run')
    for id in store_id_list:
        for device in device_id_list:
            for m in get_month_list():
                # print id, device, m[0] 
                try:
                    app_count="""SELECT app_id
                                FROM plproxy.execute_select_nestloop($$
                            	select count(distinct app_id) from mu.app_{granularity}_{device_id}_{store_id}_{month}$$) t (app_id BIGINT)""".format(device_id=device,store_id=id, month=m[1], granularity=_granularity)
                    # print one_app_sql
                    granularity_ = _granularity.capitalize()
                    app_result = pg_settings(urn, app_count, granularity_,'usage')
                    # print app_result
                    
                    
                    l = [int(x[0]) for x in app_result[granularity_]]
                    if sum(l)==0:
                        pass
                    else:
                        transform_data_path=path.format(raw_granularity=_granularity, raw_date=last_day_of_month(m[2]))
                        transform_count=spark.read.parquet(transform_data_path).filter("device_code='{}' and country_code='{}'".format(device_id_list[device],store_id_list[id] )).count()
                        print id, device, m[0] ,sum(l) , transform_count
                        t.assertEqual(sum(l),transform_count , "raw : transform data is not equal: {} ~ {}, store id :{} , device: {}, month: {} ".format(sum(l), transform_count , id, device, m[0] ))
                except psycopg2.ProgrammingError : 
                    pass
                # except Exception as e: 
                #     traceback.print_exc()
                #     print(e)
                    # traceback.print_exc()
                    # print(e)
                    #  print 'the table is not existed - granularity {}, device:{}  store_id:{} month:{}'.format(granularity_, device, id, m[0])
                
check_au_app_transform_data(IOS_COUNTRY_ID_CODES, get_device_list('ios'),'daily')
check_au_app_transform_data(ANDROID_COUNTRY_ID_CODES, get_device_list('android'),'daily')
check_au_app_transform_data(IOS_COUNTRY_ID_CODES, get_device_list('ios'),'weekly')
check_au_app_transform_data(ANDROID_COUNTRY_ID_CODES, get_device_list('android'),'weekly')
# check_au_app_transform_data(IOS_COUNTRY_ID_CODES, get_device_list('ios'),'monthly')
# check_au_app_transform_data(ANDROID_COUNTRY_ID_CODES, get_device_list('android'),'monthly')
print 'pass'



In [0]:
%%sh

PGPASSWORD='2mHdFW6%#REu' psql -h internal-aa-prod-plproxy-internal-2-31838298.us-east-1.elb.amazonaws.com -U app_bdp_usage_qa -d cohort -p 7432 << EOF  > /tmp/count_mu.txt

SELECT app_id, date
FROM plproxy.execute_select_nestloop(\$proxy\$
	select date, count(distinct app_id) from mu.app_monthly_1001_1000_201305\$proxy\$) t (app_id BIGINT, date DATE) group by date,app_id;




-- SELECT device_id,store_id,date,app_id,kpi,estimate,category_id,rank
-- FROM plproxy.execute_select_nestloop (\$proxy\$
-- select device_id,store_id,date,app_id,kpi,estimate, category_id,rank from mu.category_daily_2001_143445_201802
-- \$proxy\$) t (device_id SMALLINT,store_id INT,,app_id BIGINT,kpi SMALLINT, estimate FLOAT, category_id INT, rank INT) where app_id=100 and date='2018-02-28' and device_id='2001' limit 20;


-- SELECT device_id,store_id,date,app_id,kpi,estimate,seg_app_id
--  FROM plproxy.execute_select_nestloop(\$proxy\$select device_id,store_id,date,app_id,kpi,estimate,seg_app_id from au.app_monthly_1001_1_201802
-- \$proxy\$) t (device_id SMALLINT,store_id INT,date DATE,app_id BIGINT,kpi SMALLINT, estimate FLOAT, seg_app_id BIGINT) where app_id='20600003808593' and seg_app_id='20600000011090' order by kpi asc limit 30;

EOF


In [0]:
%python
from itertools import islice

count=0
with open('/tmp/count_mu.txt') as f:
    for line in islice(f, 2, 10):
        count=count+int(line)
print count


In [0]:

# raw_path_1="s3://b2c-prod-data-pipeline-unified-usage/unified/app-tech.usage.legacy-ag_app.v1/fact/granularity=monthly/date=2013-02/"
# spark.read.parquet(raw_path_1).printSchema()

# print spark.read.parquet(raw_path_1).filter("country_code='WW' and device_code='android-phone'").count()
# granularity='monthly'
# GRANULARIY_LIST=['monthly']
# for granularity in GRANULARIY_LIST:
#     df_date = spark.read.parquet("s3://b2c-prod-data-pipeline-unified-usage/unified/app-tech.usage.legacy-ag_app.v1/fact/granularity={}/".format(granularity)).select('date').dropDuplicates()
#     collect_date= df_date.collect()
#     print collect_date
#     date_list = [(x[0], x[0][:7]) for x in collect_date]
#     print date_list
       
    


In [0]:
%%sh
# aws s3 ls s3://b2c-prod-data-pipeline-unified-usage/unified/app-tech.usage.legacy-ad_app.v1/fact/granularity=monthly/month=2013-01/device_id=1001/store_id=1/
# aws s3 ls s3://b2c-prod-data-pipeline-unified-usage/unified/app-tech.usage.audience-demographics.v1/fact/granularity=monthly/date=2013-01-31/
# aws s3 ls s3://b2c-prod-data-pipeline-unified-usage/unified/app-tech.usage.segmented-by-age-gender.v1/fact/granularity=weekly/date=2018-12-31
# s3://b2c-prod-data-pipeline-unified-usage/unified/app-tech.usage.legacy-ag_app.v1/fact/granularity=weekly/month=2016-07/device_id=2001/store_id=0
aws s3 ls s3://b2c-prod-data-pipeline-unified-usage/unified/app-tech.usage.legacy-ag_app.v1/fact/granularity=monthly/month=2018-12/device_id=2001/


In [0]:
%python
import datetime
from dateutil.relativedelta import relativedelta

def last_day_of_month(any_day):
    next_month = any_day.replace(day=28) + datetime.timedelta(days=4)  # this will never fail
    return next_month - datetime.timedelta(days=next_month.day)


def get_month_list():
    result = []
    today = datetime.date(2019, 10, 1) 
    current = datetime.date(2018, 1, 1)    
    while current <= today:
        month_data_raw=datetime.datetime.strftime(current,'%Y-%m')
        month_data_leg_db=datetime.datetime.strftime(current,'%Y%m')
        result.append((month_data_raw, month_data_leg_db,current ))
        current += relativedelta(months=1)    
        
    return result
    

def get_every_date():
    result = []
    today = datetime.date(2019, 10, 1) 
    current = datetime.date(2018, 1, 1)    
    delta = today - current
    return [current + datetime.timedelta(days=i) for i in range(delta.days + 1)]
         
        
print get_every_date()
print get_month_list()


In [0]:
%python
